In [2]:
import pandas as pd

# Load datasets
fire = pd.read_csv("8d76ac6b-5ae8-4428-82a4-043130d17b02.csv")
crime = pd.read_csv("e03a89dd-134a-4ee8-a2bd-62c40aeebc6f.csv")
firearm = pd.read_csv("e967381d-d7e9-48e3-a2a2-39262f7fa5c4.csv")
pop = pd.read_csv("82f29015-6905-4b1c-8300-afe9bb2231b3.csv")

# Detect columns
def detect_column(df, target):
    return next((col for col in df.columns if target in col.lower()), None)

# --- Fire incidents ---
fire_neigh = detect_column(fire, "neigh")
fire_counts = fire[fire_neigh].value_counts().reset_index()
fire_counts.columns = ["Neighborhood", "Fire_Incidents"]

# --- Crime incidents ---
crime_neigh = detect_column(crime, "neigh")
crime_counts = crime[crime_neigh].value_counts().reset_index()
crime_counts.columns = ["Neighborhood", "Crime_Incidents"]

# --- Firearm seizures ---
firearm_group = firearm.groupby("neighborhood", dropna=False, as_index=False).sum()
firearm_counts = firearm_group[["neighborhood", "total_count"]].rename(columns={
    "neighborhood": "Neighborhood",
    "total_count": "Firearm_Incidents"
})

# --- Population ---
pop_neigh = detect_column(pop, "neigh")
pop_est = detect_column(pop, "estimate")
pop_data = pop[[pop_neigh, pop_est]].dropna().rename(columns={
    pop_neigh: "Neighborhood",
    pop_est: "Population"
})

# --- Merge all ---
merged = pop_data.merge(fire_counts, on="Neighborhood", how="left")
merged = merged.merge(crime_counts, on="Neighborhood", how="left")
merged = merged.merge(firearm_counts, on="Neighborhood", how="left")
merged = merged.fillna(0)

# --- Per capita calculations ---
merged["Fire_per_capita"] = merged["Fire_Incidents"] / merged["Population"]
merged["Crime_per_capita"] = merged["Crime_Incidents"] / merged["Population"]
merged["Firearm_per_capita"] = merged["Firearm_Incidents"] / merged["Population"]

# --- Safest neighborhoods ---
safest_fire = merged.sort_values("Fire_per_capita").head(10)
safest_crime = merged.sort_values("Crime_per_capita").head(10)
safest_firearm = merged.sort_values("Firearm_per_capita").head(10)

print("🔥 Top 10 Safest Neighborhoods by Fire Incidents (Per Capita):")
print(safest_fire[["Neighborhood", "Fire_per_capita"]].to_string(index=False))

print("\n🚨 Top 10 Safest Neighborhoods by Crime Incidents (Per Capita):")
print(safest_crime[["Neighborhood", "Crime_per_capita"]].to_string(index=False))

print("\n🔫 Top 10 Safest Neighborhoods by Firearm Seizures (Per Capita):")
print(safest_firearm[["Neighborhood", "Firearm_per_capita"]].to_string(index=False))

🔥 Top 10 Safest Neighborhoods by Fire Incidents (Per Capita):
        Neighborhood  Fire_per_capita
Mount Oliver Borough         0.002059
            Westwood         0.008199
       Regent Square         0.008712
         Morningside         0.011649
             Oakwood         0.012287
 Squirrel Hill North         0.012526
       Highland Park         0.012628
      Swisshelm Park         0.012639
     Stanton Heights         0.013443
       North Oakland         0.014164

🚨 Top 10 Safest Neighborhoods by Crime Incidents (Per Capita):
        Neighborhood  Crime_per_capita
          Mt. Oliver          0.000000
Mount Oliver Borough          0.000000
   Central Northside          0.007953
 Squirrel Hill North          0.025582
      Swisshelm Park          0.031970
       Regent Square          0.035818
       New Homestead          0.040082
     Stanton Heights          0.046839
       North Oakland          0.047386
 Squirrel Hill South          0.050617

🔫 Top 10 Safest Neighborho